In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

## Emissions, Population, GDP

In [2]:
# import target, pop, and gdp data per state
y_df = pd.read_csv('data/climatewatch-usemissions.csv', usecols=['State', 'Year', 'Population (People)','State GDP (Million US$ (chained 1997/2005))','Transportation (MtCO2e)'])
y_df.head()

,State,Year,Transportation (MtCO2e),State GDP (Million US$ (chained 1997/2005)),Population (People)
0,Alabama,1990,29.582733,83766.0,4050055
1,Alaska,1990,12.300752,28772.0,553290
2,Arizona,1990,24.064867,81606.0,3684097
3,Arkansas,1990,16.882259,44496.0,2356586
4,California,1990,212.571891,906103.0,29959515


In [3]:
y_df.count()

State                                          1508
Year                                           1508
Transportation (MtCO2e)                        1508
State GDP (Million US$ (chained 1997/2005))    1508
Population (People)                            1508
dtype: int64

## Transit Ridership, Gasoline Usage, Vehicle Miles Traveled (VMT), Vehicles

In [4]:
# import gas usage, transit ridership, vehicle miles traveled, and vehicle data per state
transit_df = pd.read_csv('data/transportation_usage.csv', encoding = 'utf-16', sep='\t')
transit_df.head()

,State,Measures,Year,Values
0,United States,Transit Ridership,2019,9.879421e+09
1,United States,Highway use of gasoline (thousand gallons),2019,1.360782e+08
2,United States,Highway vehicle-miles traveled (millions),2019,3.261772e+06
3,United States,Vehicles,2019,2.764912e+08
4,United States,Transit Ridership,2018,9.862598e+09


In [5]:
### Create a column for each measure in transit_df['Measures'] and assign the appropriate values
# Grab measure names
measures_list = transit_df['Measures'].unique()

# Limit list of measure names to gas usage, transit ridership, vehicle miles traveled, and vehicle data
measures_list = measures_list[0:4]

# Create initial dataframe based on transit ridership per state and per year
to_merge_df = transit_df.loc[transit_df['Measures']==measures_list[0]]
new_column_name = to_merge_df.iloc[0]['Measures']
to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
to_merge_df = to_merge_df.drop(columns=['Measures'])
transit_measures_df = to_merge_df

# Merge additional measures (gas usage, vehicle miles traveled, and vehicle data)
for column in measures_list[1:4]:
    to_merge_df = transit_df.loc[transit_df['Measures']== column]
    new_column_name = to_merge_df.iloc[0]['Measures']
    to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
    to_merge_df = to_merge_df.drop(columns=['Measures'])
    transit_measures_df = transit_measures_df.merge(to_merge_df, how = 'left', on = ['State', 'Year'])
transit_measures_df.head()

,State,Year,Transit Ridership,Highway use of gasoline (thousand gallons),Highway vehicle-miles traveled (millions),Vehicles
0,United States,2019,9.879421e+09,136078199.0,3261771.663,276491174.0
1,United States,2018,9.862598e+09,135336713.0,3240326.543,273602100.0
2,United States,2017,1.006252e+10,135300642.0,3212347.311,272480899.0
3,United States,2016,1.036766e+10,135559291.0,3174408.000,268799083.0
4,United States,2015,1.049588e+10,132242542.0,3095372.701,263610219.0


In [6]:
# Remove null values (post csv exploration)
transit_measures_df = transit_measures_df.loc[transit_measures_df['Year'] >= 1994]
transit_measures_df = transit_measures_df.loc[transit_measures_df['State'] != 'Puerto Rico']
transit_measures_df.count()

State                                         1351
Year                                          1351
Transit Ridership                             1351
Highway use of gasoline (thousand gallons)    1351
Highway vehicle-miles traveled (millions)     1351
Vehicles                                      1351
dtype: int64

In [7]:
# merge y_df and transit_measures_df
final_df = y_df.merge(transit_measures_df, on = ['State', 'Year'])
final_df.head()

,State,Year,Transportation (MtCO2e),State GDP (Million US$ (chained 1997/2005)),Population (People),Transit Ridership,Highway use of gasoline (thousand gallons),Highway vehicle-miles traveled (millions),Vehicles
0,Alabama,1994,32.389399,94803.0,4260229,9.682137e+06,2220444.0,48956.0,3176560.0
1,Alaska,1994,11.402707,26188.0,603308,3.097134e+06,273319.0,4150.0,533496.0
2,Arizona,1994,27.085805,104104.0,4245089,5.289982e+07,1899942.0,38774.0,2813460.0
3,Arkansas,1994,19.010329,53641.0,2494019,4.535502e+06,1286100.0,24948.0,1566840.0
4,California,1994,206.555572,904778.0,31484435,1.130728e+09,12932907.0,271943.0,22338870.0


In [8]:
final_df.count()

State                                          1274
Year                                           1274
Transportation (MtCO2e)                        1274
State GDP (Million US$ (chained 1997/2005))    1274
Population (People)                            1274
Transit Ridership                              1274
Highway use of gasoline (thousand gallons)     1274
Highway vehicle-miles traveled (millions)      1274
Vehicles                                       1274
dtype: int64

## Land Area (sq mi)

In [9]:
# Scrape land area (in sq mi) from:
# https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area
wikiurl = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area#cite_note-2010census-2'

# Check response code to ensure ability to download
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})
wiki_area_df = pd.read_html(str(indiatable))

# convert list to dataframe
wiki_area_df = pd.DataFrame(wiki_area_df[0])

# Drop unwanted level
wiki_area_df.columns = wiki_area_df.columns.droplevel(level=0)
wiki_area_df.head()

# Grab desired column (sq mi in index 5)
area_df = wiki_area_df.iloc[:, [0,5]]
area_df.head()

,State,sq mi
0,Alaska,570640.95
1,Texas,261231.71
2,California,155779.22
3,Montana,145545.80
4,New Mexico,121298.15


In [11]:
# manually add a value for sq mi total for US
us_df = wiki_area_df.loc[wiki_area_df['State']=='50 states and District of Columbia']
us_sqmi_df = us_df.iloc[[0], [0, 5]]
us_sqmi_df['State'] = us_sqmi_df['State'].replace({'50 states and District of Columbia': 'United States'})
area_df = area_df.append(us_sqmi_df)
area_df

,State,sq mi
0,Alaska,570640.95
1,Texas,261231.71
2,California,155779.22
3,Montana,145545.80
4,New Mexico,121298.15
...,...,...
56,Minor Outlying Islands[3][a],16.00
57,Contiguous United States,2954841.42
58,50 states and District of Columbia,3531905.43
59,"All 50 states, District of Columbia, and U.S. ...",3535948.12


In [12]:
final_df = final_df.merge(area_df, how='left', on = ['State'])
final_df

,State,Year,Transportation (MtCO2e),State GDP (Million US$ (chained 1997/2005)),Population (People),Transit Ridership,Highway use of gasoline (thousand gallons),Highway vehicle-miles traveled (millions),Vehicles,sq mi
0,Alabama,1994,32.389399,94803.0,4260229,9.682137e+06,2220444.0,4.895600e+04,3176560.0,50645.33
1,Alaska,1994,11.402707,26188.0,603308,3.097134e+06,273319.0,4.150000e+03,533496.0,570640.95
2,Arizona,1994,27.085805,104104.0,4245089,5.289982e+07,1899942.0,3.877400e+04,2813460.0,113594.08
3,Arkansas,1994,19.010329,53641.0,2494019,4.535502e+06,1286100.0,2.494800e+04,1566840.0,52035.48
4,California,1994,206.555572,904778.0,31484435,1.130728e+09,12932907.0,2.719430e+05,22338870.0,155779.22
...,...,...,...,...,...,...,...,...,...,...
1269,Washington,2018,47.607909,524486.9,7523869,2.702230e+08,2765858.0,6.236666e+04,7152413.0,66455.52
1270,West Virginia,2018,13.154700,71858.7,1804291,7.762619e+06,791672.0,1.944732e+04,1693719.0,24038.21
1271,Wisconsin,2018,31.257005,303767.4,5807406,5.896453e+07,2491234.0,6.588543e+04,5683061.0,54157.80
1272,Wyoming,2018,8.110488,38696.3,577601,2.504852e+06,294731.0,1.043844e+04,837024.0,97093.14


## Annual Temperature Highs

In [13]:
# Grab maxTemp data from https://www.ncdc.noaa.gov/cag/ for each state
# Hawaii is notably omitted
# Citation: NOAA National Centers for Environmental information, Climate at a Glance: Statewide Time Series, published June 2021, retrieved on June 14, 2021

# Create dataframe to store values for all 50 states (except Hawaii)
ncdc_temp_df = pd.DataFrame(columns=['Year', 'tempHigh', 'State'])
for i in range(50):
    if i == 48:
        continue
    state_num = str(i+1)
    ncdc_url = 'https://www.ncdc.noaa.gov/cag/statewide/time-series/'+state_num+'-tmax-12-12-1990-2018.json?base_prd=true&begbaseyear=1925&endbaseyear=2000'
    response = requests.get(ncdc_url).json()

    # Get state name
    data_title = response['description']['title']
    state_name = data_title.split(',',1)[0]

    # Rename columns to year values (omitting '12' from end)
    ncdc_temp = pd.DataFrame(response['data'])
    ncdc_temp.columns = ncdc_temp.columns.str[0:4]

    # Convert to same row/column format as final_df
    ncdc_temp = ncdc_temp.transpose()

    # Remove anomoly and create new state column
    ncdc_temp = ncdc_temp.drop(columns='anomaly')
    ncdc_temp['State'] = state_name

    # Reset index and set new Year column to int instead of object
    ncdc_temp = ncdc_temp.reset_index()
    ncdc_temp = ncdc_temp.rename(columns={'index':'Year','value':'tempHigh'})
    ncdc_temp['Year'] = ncdc_temp['Year'].astype(int)
    ncdc_temp_df = ncdc_temp_df.append(ncdc_temp)

In [14]:
# Grab national data
ncdc_url = 'https://www.ncdc.noaa.gov/cag/national/time-series/110-tmax-12-12-1990-2018.json?base_prd=true&begbaseyear=1925&endbaseyear=2000'
response = requests.get(ncdc_url).json()

# Get state name
state_name = 'United States'

# Rename columns to year values (omitting '12' from end)
ncdc_temp = pd.DataFrame(response['data'])
ncdc_temp.columns = ncdc_temp.columns.str[0:4]

# Convert to same row/column format as final_df
ncdc_temp = ncdc_temp.transpose()

# Remove anomoly and create new state column
ncdc_temp = ncdc_temp.drop(columns='anomaly')
ncdc_temp['State'] = state_name

# Reset index and set new Year column to int instead of object
ncdc_temp = ncdc_temp.reset_index()
ncdc_temp = ncdc_temp.rename(columns={'index':'Year','value':'tempHigh'})
ncdc_temp['Year'] = ncdc_temp['Year'].astype(int)
ncdc_temp_df = ncdc_temp_df.append(ncdc_temp)

In [15]:
final_df = final_df.merge(ncdc_temp_df, how='left', on = ['State', 'Year'])
final_df

,State,Year,Transportation (MtCO2e),State GDP (Million US$ (chained 1997/2005)),Population (People),Transit Ridership,Highway use of gasoline (thousand gallons),Highway vehicle-miles traveled (millions),Vehicles,sq mi,tempHigh
0,Alabama,1994,32.389399,94803.0,4260229,9.682137e+06,2220444.0,4.895600e+04,3176560.0,50645.33,74.1
1,Alaska,1994,11.402707,26188.0,603308,3.097134e+06,273319.0,4.150000e+03,533496.0,570640.95,33.7
2,Arizona,1994,27.085805,104104.0,4245089,5.289982e+07,1899942.0,3.877400e+04,2813460.0,113594.08,74.7
3,Arkansas,1994,19.010329,53641.0,2494019,4.535502e+06,1286100.0,2.494800e+04,1566840.0,52035.48,71.1
4,California,1994,206.555572,904778.0,31484435,1.130728e+09,12932907.0,2.719430e+05,22338870.0,155779.22,70.6
...,...,...,...,...,...,...,...,...,...,...,...
1269,Washington,2018,47.607909,524486.9,7523869,2.702230e+08,2765858.0,6.236666e+04,7152413.0,66455.52,57.7
1270,West Virginia,2018,13.154700,71858.7,1804291,7.762619e+06,791672.0,1.944732e+04,1693719.0,24038.21,63.3
1271,Wisconsin,2018,31.257005,303767.4,5807406,5.896453e+07,2491234.0,6.588543e+04,5683061.0,54157.80,53.1
1272,Wyoming,2018,8.110488,38696.3,577601,2.504852e+06,294731.0,1.043844e+04,837024.0,97093.14,55.5


In [16]:
# Export to CSV
final_df.to_csv("resources/annualAllStateData.csv", index = False, header = True)

## Number of Alternative Fueling Stations

In [17]:
fuel_df = pd.read_csv('data/altFuelStations.csv', usecols=['Fuel Type Code', 'State', 'Open Date'])
fuel_df.head()

C:\Users\Nick\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Fuel Type Code,State,Open Date
0,CNG,GA,1994-07-15
1,CNG,TX,1996-12-15
2,CNG,AR,1997-01-01
3,CNG,MA,1996-11-15
4,CNG,MA,1996-11-15


In [18]:
# Convert Open Date to Year
fuel_df['Open Date'] = fuel_df['Open Date'].str[:4]

# Drop null values
fuel_df = fuel_df.dropna()

# Replace state abbreviations with full names
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

fuel_df['State'].replace(states, inplace=True)

In [19]:
# Group by state and year to get counts of stations per state per year
fuel_df['stateYear'] = fuel_df['State'] + fuel_df['Open Date']
fuel_df
fuel_df_counts = fuel_df.groupby(['stateYear'])
fuel_df_counts = fuel_df_counts.count()
fuel_df_counts = fuel_df_counts.reset_index()
fuel_df_counts

,stateYear,Fuel Type Code,State,Open Date
0,AB1987,3,3,3
1,AB1989,1,1,1
2,AB1990,1,1,1
3,AB1992,2,2,2
4,AB1995,1,1,1
...,...,...,...,...
982,Wyoming2018,11,11,11
983,Wyoming2019,3,3,3
984,Wyoming2020,3,3,3
985,Wyoming2021,1,1,1


In [20]:
# Unpack state and year columns
fuel_df_counts['Year'] = fuel_df_counts['stateYear'].str[-4:]
fuel_df_counts['State'] = fuel_df_counts['stateYear'].str[:-4]

In [21]:
# Create final counts dataframe for merge to final_df
fuel_df_counts_final = fuel_df_counts[['State', 'Year', 'Open Date']]
fuel_df_counts_final = fuel_df_counts_final.rename(columns={'Open Date':'altFuelStations'})
fuel_df_counts_final

,State,Year,altFuelStations
0,AB,1987,3
1,AB,1989,1
2,AB,1990,1
3,AB,1992,2
4,AB,1995,1
...,...,...,...
982,Wyoming,2018,11
983,Wyoming,2019,3
984,Wyoming,2020,3
985,Wyoming,2021,1


In [22]:
type(fuel_df_counts_final['State'])

pandas.core.series.Series

In [23]:
test_df = final_df.merge(fuel_df_counts_final, how='left', on = ['State', 'Year'])
test_df

,State,Year,Transportation (MtCO2e),State GDP (Million US$ (chained 1997/2005)),Population (People),Transit Ridership,Highway use of gasoline (thousand gallons),Highway vehicle-miles traveled (millions),Vehicles,sq mi,tempHigh,altFuelStations
0,Alabama,1994,32.389399,94803.0,4260229,9.682137e+06,2220444.0,4.895600e+04,3176560.0,50645.33,74.1,NaN
1,Alaska,1994,11.402707,26188.0,603308,3.097134e+06,273319.0,4.150000e+03,533496.0,570640.95,33.7,NaN
2,Arizona,1994,27.085805,104104.0,4245089,5.289982e+07,1899942.0,3.877400e+04,2813460.0,113594.08,74.7,NaN
3,Arkansas,1994,19.010329,53641.0,2494019,4.535502e+06,1286100.0,2.494800e+04,1566840.0,52035.48,71.1,NaN
4,California,1994,206.555572,904778.0,31484435,1.130728e+09,12932907.0,2.719430e+05,22338870.0,155779.22,70.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1269,Washington,2018,47.607909,524486.9,7523869,2.702230e+08,2765858.0,6.236666e+04,7152413.0,66455.52,57.7,NaN
1270,West Virginia,2018,13.154700,71858.7,1804291,7.762619e+06,791672.0,1.944732e+04,1693719.0,24038.21,63.3,NaN
1271,Wisconsin,2018,31.257005,303767.4,5807406,5.896453e+07,2491234.0,6.588543e+04,5683061.0,54157.80,53.1,NaN
1272,Wyoming,2018,8.110488,38696.3,577601,2.504852e+06,294731.0,1.043844e+04,837024.0,97093.14,55.5,NaN


In [24]:
#final_df.count()